In [1]:
import os
import eikon as ek
import numpy as np
import pickle
import pandas as pd
import cufflinks as cf
import configparser as cp

In [2]:
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')  # adjust for different file location

['eikon.cfg']

In [3]:
ek.set_app_key(cfg['eikon']['app_id']) #set_app_id function being deprecated

## Get a List of CIKs

### From Compustat
#### For Both Bankrupt and Healthy Companies

In [4]:
# set the path to csv files
# path = 'D:\\studyproject\\bankruptcy\\data\\Compustat\\' # for win
path = '/Users/user/Documents/Bankruptcy/bankruptcy/data/Compustat/' # for mac

# read the csv files
bankrupt_compustat = pd.read_csv(path + 'list_bankrupt.csv', dtype=object, index_col=0)
healthy = pd.read_csv(path + 'list_healthy.csv', dtype=object, index_col=0)

bankrupt_compustat.head()

,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK
0,001367,Amber Resources Company of C,08/31/2012,2.0,3AMBE,023184203,0000276750
1,002033,Fairchild Corp. (The),11/01/2011,2.0,FCHDQ,303698104,0000009779
2,004049,Constar International Inc,06/01/2011,2.0,CNSTQ,21036U206,0000029806
3,004352,Energy Conversion Devices In,09/04/2012,2.0,ENERQ,292659109,0000032878
4,004768,Fleetwood Enterprises Inc.,08/23/2010,2.0,FLTWQ,339099103,0000314132


In [5]:
healthy.head()

,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK
0,001004,AAR Corp,.,NaN,AIR,000361105,0000001750
1,001013,ADC Telecommunications Inc.,12/10/2010,1.0,ADCT,000886309,0000061478
2,001021,AFP Imaging Corp,09/15/2014,7.0,IWKS,001058205,0000319126
3,001034,Alpharma Inc.,12/31/2008,1.0,ALO.2,020813101,0000730469
4,001045,American Airlines Group Inc,.,NaN,AAL,02376R102,0000006201


In [6]:
bankrupt_compustat_CIK = bankrupt_compustat.CIK.dropna().to_list()
healthy_CIK = healthy.CIK.dropna().to_list()

print(len(bankrupt_compustat_CIK), 'CIKs available for Compustat bankrupt companies, and')
print(len(healthy_CIK), 'CIKs available for healthy companies.')


100 CIKs available for Compustat bankrupt companies, and
14761 CIKs available for healthy companies.


### From Edgar
#### For Bankrupt Companies

In [7]:
# set the path to csv file
# path = 'D:\\studyproject\\bankruptcy\\data\\Edgar\\' # for win
path = '/Users/user/Documents/Bankruptcy/bankruptcy/data/Edgar/' # for mac

# read the csv files
bankrupt_edgar = pd.read_csv(path + 'edgar_bankrupt_companies.csv', dtype=object)

bankrupt_edgar.head()

,CIK,Company name,Bankruptcy chapter,Form type,Date,URL
0,0000002034,ACETO CORP,11,8-K,09/20/2019,https://www.sec.gov/Archives/edgar/data/2034/0...
1,0000002178,"ADAMS RESOURCES & ENERGY, INC.",11,8-K,04/24/2017,https://www.sec.gov/Archives/edgar/data/2178/0...
2,0000004515,AMERICAN AIRLINES INC,11,8-K,11/29/2011,https://www.sec.gov/Archives/edgar/data/4515/0...
3,0000004707,AMERICA CAPITAL CORP,11,8-K,02/09/2009,https://www.sec.gov/Archives/edgar/data/4707/0...
4,0000006201,AMR CORP,11,8-K,10/23/2013,https://www.sec.gov/Archives/edgar/data/6201/0...


In [8]:
bankrupt_edgar_CIK = bankrupt_edgar.CIK.dropna().to_list()

print(len(bankrupt_edgar_CIK), 'CIKs available for Edgar bankrupt companies.')

1075 CIKs available for Edgar bankrupt companies.


## Merging Bankrupt Lists

In [9]:
# add two lists and remove the duplicates
bankrupt_CIK = list(set(bankrupt_compustat_CIK + bankrupt_edgar_CIK))

len(bankrupt_CIK)

1103

****

## Save CIK Lists

In [10]:
# pickle down the bankrupt CIK list
with open('CIK_list_bankrupt.txt', 'wb') as f:
    pickle.dump(bankrupt_CIK, f)

# pickle down the healthy CIK list
with open('CIK_list_healthy.txt', 'wb') as f:
    pickle.dump(healthy_CIK, f)

****

## Download the Data

In [11]:
# create folders for saving data
if not os.path.exists('data_bankrupt_CIK'):
    os.makedirs('data_bankrupt_CIK')
    
if not os.path.exists('data_healthy_CIK'):
    os.makedirs('data_healthy_CIK')

In [12]:
# Eikon does not work with large lists, so we
# need to break the healthy_CIK list

healthy_CIK_list = [healthy_CIK[i:i + 1000] for i in range(0, len(healthy_CIK), 1000)]

len(healthy_CIK_list)


15

### Historical Data

In [ ]:
# define a function for getting timeseries
'''
def timeseries(cik_list, folder):
    '''
    the function gets a list of CIKs and name of a folder,
    and saves a dataframe of historical data for each valid
    CIK in the list, in that folder.
    '''
    for cik in cik_list:
        try:

            df = ek.get_timeseries(cik,  # the CIK for the company
                                 fields='*',  # all fields
                                 start_date='2000-01-01',  # start date
                                 end_date='2019-08-19')  # end date
            df.to_csv(folder + '/{}.csv'.format(cik))
            
        except:
            pass

        

# get timeseries for bankrupt companies
timeseries(bankrupt_CIK, 'data_bankrupt_CIK')
'''


In [ ]:
# and healthy ones
# ----------------------------------------------------
# for eikon data-download limitation I write the code
# for each list in a separate line, and you know what,
# in a separate cell, as well. This way we could resume
# downloading after 12 hours without any unnecessary
# confusion.
# ----------------------------------------------------

# 1 out of 8 - undone
# timeseries(healthy_CIK_list[0], 'data_healthy_CIK')


In [ ]:
# 2 out of 8 - undone
# timeseries(healthy_CIK_list[1], 'data_healthy_CIK')


In [ ]:
# 3 out of 8 - undone
# timeseries(healthy_CIK_list[2], 'data_healthy_CIK')


In [ ]:
# 4 out of 8 - undone
# timeseries(healthy_CIK_list[3], 'data_healthy_CIK')


In [ ]:
# 5 out of 8 - undone
# timeseries(healthy_CIK_list[4], 'data_healthy_CIK')


In [ ]:
# 6 out of 8 - undone
# timeseries(healthy_CIK_list[5], 'data_healthy_CIK')


In [ ]:
# 7 out of 8 - undone
# timeseries(healthy_CIK_list[6], 'data_healthy_CIK')


In [ ]:
# 8 out of 8 - undone
# timeseries(healthy_CIK_list[7], 'data_healthy_CIK')


### Financial Factors


In [41]:
# add all financial factors we need to replicate *the paper*

field1 = ['TR.TotalCurrentAssets', # Current Assets
          'TR.TotalCurrLiabilities', # Current Liabilities
          'TR.AccountsPayableCFStmt', # Accounts Payable
          'TR.NetSales', # 
          'TR.CashAndSTInvestments',
          'TR.TotalAssetsReported',
          'TR.CashAndSTInvestments',
          'TR.MarketCapDS',
          'TR.TotalCurrLiabilities',
          'TR.Cash',
          'TR.TotalAssetsReported',
          'TR.Cash',
          'TR.TotalCurrLiabilities',
          'TR.BGS.BusEarningsBeforeInterestandTaxesEbit',
          'TR.BGS.GeoEarningsBeforeInterestandTaxesEbit',
          'TR.DepreciationAmort',
          'TR.TotalAssetsReported',
          'TR.BGS.BusEarningsBeforeInterestandTaxesEbit',
          'TR.BGS.GeoEarningsBeforeInterestandTaxesEbit',
          'TR.TotalAssetsReported',
          'TR.BGS.BusEarningsBeforeInterestandTaxesEbit',
          'TR.BGS.GeoEarningsBeforeInterestandTaxesEbit',
          'TR.NetSales',
          'TR.TotalDebtOutstanding',
          'TR.NetDebt',
          'TR.TotalAssetsReported',
          'TR.PCTotInvt5YrCompAnnGrowthRate',
          'TR.Inventories',
          'TR.Inventories',
          'TR.NetSales',
          'TR.TotalCurrLiabilities',
          'TR.Cash']

field2 = ['TR.TotalAssetsReported',
          'TR.TotalCurrLiabilities',
          'TR.TotalAssetsReported',
          'TR.TotalCurrLiabilities',
          'TR.TotalLiabilities',
          'TR.TotalCurrLiabilities',
          'TR.NetSales',
          'TR.TotalLiabilities',
          'TR.TotalAssetsReported',
          'TR.TotalLiabilities',
          'TR.TotalLiabilities',
          'TR.TangibleBookValueRptd',
          'TR.TangibleBookValueBnk',
          'TR.TangibleBookValueIns',
          'TR.TangibleBookValueUtil',
          'TR.TangibleBVPS',
          'TR.NetIncomeStartingLine',
          'TR.TotalAssetsReported',
          'TR.NetIncomeStartingLine']

field3 = ['TR.NetIncomeStartingLine',
          'TR.NetSales',
          'TR.OperatingIncome',
          'TR.TotalAssetsReported',
          'TR.STOCKPRICE',
          'TR.FiPrice',
          'TR.TotalCurrLiabilities',
          'TR.RetainedEarnings',
          'TR.TotalAssetsReported',
          'TR.MARKETCAPITALISATION',
          'MKT_CAP',
          'TR.TotalEquity']

field4 = ['TR.TotalAssetsReported',
          'TR.FiPricingVolatility',
          'TR.PCWrkgCapPrdOverPrdAvg(Period=FY0)',
          'TR.PCWrkgCapPrdOverPrdPctChg(Period=FY0)',
          'TR.PZPHasFiledForBankruptcy', # bankruptcy check (not working)
          'TR.PZPBankruptcyFilingDate', # bankruptcy check (not working)
          'TR.InstrumentIsActive', # bankruptcy check (not reliable)
          'TR.IsDelistedQuote',
          'TR.GICSIndustry',
          'TR.NAICSIndustryGroup']  # bankruptcy check (not reliable)

# remove duplicates
field1 = list(set(field1))
field2 = list(set(field2))
field3 = list(set(field3))
field4 = list(set(field4))

print(len(field1))
print(len(field2))
print(len(field3))
print(len(field4))

15
10
11
10


### Bankrupt Companies

In [ ]:
# download the financial factors for the first list of 

# for the bankrupt companies
# get the data
bankrupt_data, err = ek.get_data(bankrupt_CIK, fields=field1,
                                 parameters={'Frq':'Q',
                                             'SDate':'2000-01-01',
                                             'EDate':'2019-06-30'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1.csv')


In [ ]:
# download the financial factors

# for the bankrupt companies
# get the data
bankrupt_data, err = ek.get_data(bankrupt_CIK, fields=field2,
                                 parameters={'Frq':'Q',
                                             'SDate':'2000-01-01',
                                             'EDate':'2019-06-30'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2.csv')


In [42]:
# download the financial factors

# for the bankrupt companies
# get the data
bankrupt_data, err = ek.get_data(bankrupt_CIK, fields=field3,
                                 parameters={'Frq':'Q',
                                             'SDate':'2000-01-01',
                                             'EDate':'2019-06-30'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3.csv')


In [43]:
# download the financial factors

# for the bankrupt companies
# get the data
bankrupt_data, err = ek.get_data(bankrupt_CIK, fields=field4,
                                 parameters={'Frq':'Q',
                                             'SDate':'2000-01-01',
                                             'EDate':'2019-06-30'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data4.csv')


****

### Healthy Companies

In [14]:
# again for the healthy list we need to arrange
# download precedure in different cells

# chunk 1 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[0], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk01_fields1.csv')


In [15]:
# again for the healthy list we need to arrange
# download precedure in different cells

# chunk 1 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[0], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk01_fields2.csv')


In [44]:
# again for the healthy list we need to arrange
# download precedure in different cells

# chunk 1 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[0], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk01_fields3.csv')


In [45]:
# again for the healthy list we need to arrange
# download precedure in different cells

# chunk 1 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[0], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk01_fields4.csv')


****

In [17]:
# chunk 2 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[1], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk02_fields1.csv')


In [18]:
# chunk 2 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[1], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk02_fields2.csv')


In [46]:
# chunk 2 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[1], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk02_fields3.csv')


In [47]:
# chunk 2 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[1], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk02_fields4.csv')



****

In [21]:
# chunk 3 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[2], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk03_fields1.csv')


In [22]:
# chunk 3 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[2], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk03_fields2.csv')


In [48]:
# chunk 3 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[2], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk03_fields3.csv')


In [49]:
# chunk 3 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[2], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk03_fields4.csv')


****

In [24]:
# chunk 4 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[3], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk04_fields1.csv')


In [25]:
# chunk 4 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[3], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk04_fields2.csv')


In [50]:
# chunk 4 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[3], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk04_fields3.csv')


In [51]:
# chunk 4 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[3], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk04_fields4.csv')


****

In [27]:
# chunk 5 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[4], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk05_fields1.csv')


In [28]:
# chunk 5 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[4], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk05_fields2.csv')


In [52]:
# chunk 5 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[4], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk05_fields3.csv')


In [53]:
# chunk 5 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[4], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk05_fields4.csv')


****

In [30]:
# chunk 6 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[5], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk06_fields1.csv')


In [31]:
# chunk 6 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[5], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk06_fields2.csv')


In [54]:
# chunk 6 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[5], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk06_fields3.csv')


In [55]:
# chunk 6 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[5], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk06_fields4.csv')



****

In [33]:
# chunk 7 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[6], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk07_fields1.csv')


In [34]:
# chunk 7 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[6], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk07_fields2.csv')


In [56]:
# chunk 7 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[6], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk07_fields3.csv')


In [57]:
# chunk 7 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[6], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk07_fields4.csv')


****

In [36]:
# chunk 8 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[7], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk08_fields1.csv')


In [37]:
# chunk 8 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[7], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk08_fields2.csv')


In [58]:
# chunk 8 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[7], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk08_fields3.csv')


In [59]:
# chunk 8 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[7], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk08_fields4.csv')



****

In [ ]:
# chunk 9 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[8], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk09_fields1.csv')


In [ ]:
# chunk 9 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[8], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk09_fields2.csv')


In [ ]:
# chunk 9 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[8], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk09_fields3.csv')


In [ ]:
# chunk 9 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[8], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk09_fields4.csv')


****

In [ ]:
# chunk 10 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[9], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk10_fields1.csv')


In [ ]:
# chunk 10 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[9], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk10_fields2.csv')


In [ ]:
# chunk 10 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[9], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk10_fields3.csv')


In [ ]:
# chunk 10 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[9], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk10_fields4.csv')


****

In [ ]:
# chunk 11 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[10], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk11_fields1.csv')


In [ ]:
# chunk 11 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[10], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk11_fields2.csv')


In [ ]:
# chunk 11 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[10], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk11_fields3.csv')


In [ ]:
# chunk 11 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[10], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk11_fields4.csv')


****

In [ ]:
# chunk 12 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[11], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk12_fields1.csv')


In [ ]:
# chunk 12 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[11], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk12_fields2.csv')


In [ ]:
# chunk 12 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[11], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk12_fields3.csv')


In [ ]:
# chunk 12 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[11], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk12_fields4.csv')


****

In [ ]:
# chunk 13 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[12], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk13_fields1.csv')


In [ ]:
# chunk 13 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[12], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk13_fields2.csv')


In [ ]:
# chunk 13 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[12], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk13_fields3.csv')


In [ ]:
# chunk 13 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[12], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk13_fields4.csv')


****

In [ ]:
# chunk 14 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[13], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk14_fields1.csv')


In [ ]:
# chunk 14 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[13], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk14_fields2.csv')


In [ ]:
# chunk 14 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[13], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk14_fields3.csv')


In [ ]:
# chunk 14 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[13], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk14_fields4.csv')


****

In [ ]:
# chunk 15 out of 15 - fields 1 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[14], fields=field1,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk15_fields1.csv')


In [ ]:
# chunk 15 out of 15 - fields 2 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[14], fields=field2,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk15_fields2.csv')


In [ ]:
# chunk 15 out of 15 - fields 3 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[14], fields=field3,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk15_fields3.csv')


In [ ]:
# chunk 15 out of 15 - fields 4 out of 4
healthy_df, err = ek.get_data(healthy_CIK_list[14], fields=field4,
                              parameters={'Frq':'Q',
                                          'SDate':'2000-01-01',
                                          'EDate':'2019-06-30'})

healthy_df.to_csv('data_healthy_CIK/healthy_chunk15_fields4.csv')


****

## Load CSV Into 1 Dataframe

In [ ]:
# concatenate the 15 dataframes in the list
healthy_data = pd.concat([df for df in healthy_data_list])
    
# and save the merged dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data.csv')


In [ ]:
# test the resulting dataframe for bankrupt company
print(len(bankrupt_data))
bankrupt_data.head()


In [ ]:
bankrupt_data

In [ ]:
# test the resulting dataframe for healthy company
print(len(healthy_data))
healthy_data.head()


In [ ]:
healthy_data

****

In [ ]:
# use screen to find more bankrupt companies using eikon data item
# something = ek.get_data('SCREEN(IN(TR.PZPHasFiledForBankruptcy, Y))', ['TR.CommonName', 'TR.RIC])

